# Keras-RL DQN Model


At first we will import all necessary packages:

In [1]:
import time  # to reduce the game speed when playing manually

import gym  # Contains the game we want to play
from pyglet.window import key  # for manual playing

# import necessary blocks from keras to build the Deep Learning backbone of our agent
from tensorflow.keras.models import Sequential  # To compose multiple Layers
from tensorflow.keras.layers import Dense  # Fully-Connected layer
from tensorflow.keras.layers import Activation  # Activation functions
from tensorflow.keras.layers import Flatten  # Flatten function

from tensorflow.keras.optimizers import Adam  # Adam optimizer

# Now the keras-rl2 agent. Dont get confused as it is only called rl and not keras-rl

from rl.agents.dqn import DQNAgent  # Use the basic Deep-Q-Network agent

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})

wandb: Currently logged in as: skyfall-blue (use `wandb login --relogin` to force relogin)


Now we will create the environment:

In [2]:
env_name = ENV_NAME = 'LunarLander-v2'  # https://gym.openai.com/envs/LunarLander-v2/
env = gym.make(env_name)  # create the environment
nb_actions = env.action_space.n  # get the number of possible actions
NUMBER_STEPS = 150000

Lets watch how the game looks when chosing random actions

In [5]:
env.reset()  # reset the environment to the initial state
for _ in range(200):  # play for max 200 iterations
    env.render(mode="human")  # render the current game state on your screen
    random_action = env.action_space.sample()  # chose a random action
    env.step(random_action)  # execute that action
env.close()  # close the environment

In [6]:
action = 0
def key_press(k, mod):
    '''
    This function gets the key press for gym
    '''
    global action
    if k == key.A: #fly left
        action = 3
    if k == key.D: #fly right
        action = 1
    if k == key.W: #fly up
        action = 2
    if k == key.S:
        action = 0

env.reset()
rewards = 0
for _ in range(1000):
    env.render(mode="human")
    env.viewer.window.on_key_press = key_press  # update the key press
    observation, reward, done, info = env.step(action)
    rewards+=1
    if done:
        print(f"You got {rewards} points!")
        break
    time.sleep(0.05)  # reduce speed a little bit
env.close()

You got 188 points!


**TASK: Create the Neural Network for your Deep-Q-Agent**
Take a look at the size of the action space and the size of the observation space.
You are free to chose any architecture you want!
Hint: It already works with three layers, each having 64 neurons.

In [7]:
model = Sequential()
# https://keras.io/api/layers/reshaping_layers/flatten/
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(nb_actions))
model.add(Activation('linear'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                576       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0

Lets create the DQN agent from keras-rl
For this setting, the agent takes the following parameters:

1. model = The model
2. nb_actions = The number of actions (2 in this case)
3. memory = The action replay memory. You can choose between the *SequentialMemory()* and *EpisodeParameterMemory() which is only used for one RL agent called CEM*
4. nb_steps_warmup = How many iterations without training - Used to fill the memory
5. target_model_update = When do we update the target model?
6. Action Selection policy. You can choose between a *LinearAnnealedPolicy()*, *SoftmaxPolicy()*, *EpsGreedyQPolicy()*, *GreedyQPolicy()*, *GreedyQPolicy()*, *MaxBoltzmannQPolicy()* and *BoltzmannGumbelQPolicy()*. We use all of them during the next notebooks but feel free to try them out and inspect which works best here

There are some more parameters, you can pass to the DQN Agent. Feel free to explore them, but we will also take a look at them together in the remaining notebooks

Here we initialize the circular buffer with a limit of 50000 and a window length of 1.
The window length describes the number of subsequent actions stored for a state.
This will be demonstrated in the next lecture, when we start dealing with images

In [8]:
from rl.memory import SequentialMemory  # Sequential Memory for storing observations ( optimized circular buffer)

memory = SequentialMemory(limit=50000, window_length=1)


Then we define the Action Selection Policy: <br />
We use *LinearAnnealedPolicy* in order to perform the epsilon greedy strategy with decaying epsilon. <br />
*LinearAnnealedPolicy* accepts an action selection policy, its maximal and minimal values and a step number in order to create a dynimal policy. <br/>
The minimal value epsilon can reach during training is 0.1.<br />
For evaluation (e.g running the agent) it is fixed to 0


In [9]:
# LinearAnnealedPolicy allows to decay the epsilon for the epsilon greedy strategy
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps',
                              value_max=1.,
                              value_min=.1,
                              value_test=0,
                              nb_steps=NUMBER_STEPS) 


**TASK: Create the DQNAgent** <br />
Feel free to play with the nb_steps_warump, target_model_update, batch_size and gamma parameters. <br />
Hint:<br />
You can try *nb_steps_warmup*=100, *target_model_update*=1000, *batch_size*=64 and *gamma*=0.992 as a first guess

In [10]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1000, policy=policy, batch_size=64, gamma=0.992)

Finally we compile our model with the Adam optimizer and a learning rate of 0.001.<br />
We log the Mean Absolute Error

In [11]:
# Use learning_rate instead of lr if you get warning
dqn.compile(Adam(lr=0.001), metrics=['mae']) 

Now we run the training for 150000 steps. You can change visualize=True if you want to watch your model learning.
Keep in mind that this increases the running time
The training time is around 30 min so grep your favorite beverage and stay tuned


In [9]:
dqn.fit(env, nb_steps=NUMBER_STEPS, visualize=False, verbose=2, callbacks=[WandbCallback()])

Training for 150000 steps ...
     93/150000: episode: 1, duration: 0.133s, episode steps:  93, steps per second: 701, episode reward: -115.098, mean reward: -1.238 [-100.000, 12.663], mean action: 1.559 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


C:\Users\nguye\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\nguye\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    210/150000: episode: 2, duration: 1.218s, episode steps: 117, steps per second:  96, episode reward: -171.760, mean reward: -1.468 [-100.000, 19.164], mean action: 1.624 [0.000, 3.000],  loss: 24.544618, mae: 1.193189, mean_q: 1.460490, mean_eps: 0.999070
    294/150000: episode: 3, duration: 0.487s, episode steps:  84, steps per second: 172, episode reward: -80.888, mean reward: -0.963 [-100.000,  7.298], mean action: 1.583 [0.000, 3.000],  loss: 31.907632, mae: 1.597973, mean_q: 1.511593, mean_eps: 0.998491
    414/150000: episode: 4, duration: 0.715s, episode steps: 120, steps per second: 168, episode reward: -379.890, mean reward: -3.166 [-100.000, 110.786], mean action: 1.450 [0.000, 3.000],  loss: 23.614650, mae: 2.098975, mean_q: 2.325129, mean_eps: 0.997879
    506/150000: episode: 5, duration: 0.543s, episode steps:  92, steps per second: 169, episode reward: -331.912, mean reward: -3.608 [-100.000,  5.127], mean action: 1.598 [0.000, 3.000],  loss: 25.103136, mae: 2.50062

   3140/150000: episode: 34, duration: 0.662s, episode steps: 101, steps per second: 153, episode reward: -121.345, mean reward: -1.201 [-100.000, 13.871], mean action: 1.475 [0.000, 3.000],  loss: 18.427108, mae: 5.363812, mean_q: 3.543080, mean_eps: 0.981466
   3278/150000: episode: 35, duration: 0.886s, episode steps: 138, steps per second: 156, episode reward: -154.994, mean reward: -1.123 [-100.000,  7.176], mean action: 1.645 [0.000, 3.000],  loss: 16.396252, mae: 5.380090, mean_q: 3.677482, mean_eps: 0.980749
   3345/150000: episode: 36, duration: 0.487s, episode steps:  67, steps per second: 138, episode reward: -95.707, mean reward: -1.428 [-100.000, 16.914], mean action: 1.478 [0.000, 3.000],  loss: 13.288225, mae: 5.221876, mean_q: 3.907543, mean_eps: 0.980134
   3470/150000: episode: 37, duration: 0.866s, episode steps: 125, steps per second: 144, episode reward: -156.282, mean reward: -1.250 [-100.000,  5.550], mean action: 1.552 [0.000, 3.000],  loss: 18.133352, mae: 5.37

   6163/150000: episode: 66, duration: 0.461s, episode steps:  72, steps per second: 156, episode reward: -50.242, mean reward: -0.698 [-100.000, 11.999], mean action: 1.528 [0.000, 3.000],  loss: 13.400370, mae: 7.743819, mean_q: 5.035287, mean_eps: 0.963241
   6269/150000: episode: 67, duration: 0.717s, episode steps: 106, steps per second: 148, episode reward: -285.082, mean reward: -2.689 [-100.000, 108.083], mean action: 1.500 [0.000, 3.000],  loss: 11.429059, mae: 7.966436, mean_q: 4.650674, mean_eps: 0.962707
   6351/150000: episode: 68, duration: 0.640s, episode steps:  82, steps per second: 128, episode reward: -123.706, mean reward: -1.509 [-100.000,  8.191], mean action: 1.476 [0.000, 3.000],  loss: 12.435406, mae: 8.026945, mean_q: 4.363116, mean_eps: 0.962143
   6488/150000: episode: 69, duration: 1.036s, episode steps: 137, steps per second: 132, episode reward: -211.840, mean reward: -1.546 [-100.000, 14.325], mean action: 1.723 [0.000, 3.000],  loss: 15.239830, mae: 8.0

...

  79641/150000: episode: 802, duration: 1.065s, episode steps: 153, steps per second: 144, episode reward: -186.349, mean reward: -1.218 [-100.000,  8.672], mean action: 1.673 [0.000, 3.000],  loss: 4.303985, mae: 35.575775, mean_q: 20.611393, mean_eps: 0.522616
  79775/150000: episode: 803, duration: 0.912s, episode steps: 134, steps per second: 147, episode reward: -10.044, mean reward: -0.075 [-100.000, 15.200], mean action: 1.552 [0.000, 3.000],  loss: 6.506110, mae: 35.953140, mean_q: 20.787103, mean_eps: 0.521755
  79933/150000: episode: 804, duration: 1.131s, episode steps: 158, steps per second: 140, episode reward: -151.013, mean reward: -0.956 [-100.000, 41.239], mean action: 1.797 [0.000, 3.000],  loss: 4.691068, mae: 35.455677, mean_q: 21.312353, mean_eps: 0.520879
  80036/150000: episode: 805, duration: 0.691s, episode steps: 103, steps per second: 149, episode reward: -39.704, mean reward: -0.385 [-100.000, 15.872], mean action: 1.689 [0.000, 3.000],  loss: 5.614282, mae:

  85845/150000: episode: 834, duration: 0.799s, episode steps: 113, steps per second: 141, episode reward: -23.486, mean reward: -0.208 [-100.000, 21.832], mean action: 1.717 [0.000, 3.000],  loss: 8.117824, mae: 35.277742, mean_q: 21.396441, mean_eps: 0.485272
  85926/150000: episode: 835, duration: 0.552s, episode steps:  81, steps per second: 147, episode reward: -36.282, mean reward: -0.448 [-100.000, 10.965], mean action: 1.840 [0.000, 3.000],  loss: 9.446624, mae: 35.244554, mean_q: 21.342148, mean_eps: 0.484690
  86163/150000: episode: 836, duration: 1.778s, episode steps: 237, steps per second: 133, episode reward: -73.835, mean reward: -0.312 [-100.000, 15.453], mean action: 1.759 [0.000, 3.000],  loss: 7.407952, mae: 34.959829, mean_q: 21.621873, mean_eps: 0.483736
  86262/150000: episode: 837, duration: 0.657s, episode steps:  99, steps per second: 151, episode reward: -163.675, mean reward: -1.653 [-100.000,  1.490], mean action: 1.586 [0.000, 3.000],  loss: 8.212032, mae: 

  99009/150000: episode: 866, duration: 1.464s, episode steps: 209, steps per second: 143, episode reward: -44.859, mean reward: -0.215 [-100.000, 52.220], mean action: 1.579 [0.000, 3.000],  loss: 7.151001, mae: 30.051914, mean_q: 23.205500, mean_eps: 0.406576
  99307/150000: episode: 867, duration: 2.098s, episode steps: 298, steps per second: 142, episode reward: -166.036, mean reward: -0.557 [-100.000, 18.946], mean action: 1.762 [0.000, 3.000],  loss: 7.398812, mae: 30.582721, mean_q: 24.413634, mean_eps: 0.405055
  99463/150000: episode: 868, duration: 1.127s, episode steps: 156, steps per second: 138, episode reward: -55.357, mean reward: -0.355 [-100.000,  9.588], mean action: 1.641 [0.000, 3.000],  loss: 7.815472, mae: 30.394103, mean_q: 23.955935, mean_eps: 0.403693
  99688/150000: episode: 869, duration: 1.579s, episode steps: 225, steps per second: 142, episode reward: -81.323, mean reward: -0.361 [-100.000, 19.505], mean action: 1.711 [0.000, 3.000],  loss: 8.012784, mae: 

 119360/150000: episode: 898, duration: 7.605s, episode steps: 1000, steps per second: 131, episode reward: 153.488, mean reward:  0.153 [-24.100, 22.827], mean action: 1.089 [0.000, 3.000],  loss: 9.017034, mae: 23.155876, mean_q: 25.286232, mean_eps: 0.286843
 119901/150000: episode: 899, duration: 4.095s, episode steps: 541, steps per second: 132, episode reward: -204.692, mean reward: -0.378 [-100.000, 16.156], mean action: 1.750 [0.000, 3.000],  loss: 9.133801, mae: 22.828125, mean_q: 25.539530, mean_eps: 0.282220
 120901/150000: episode: 900, duration: 7.460s, episode steps: 1000, steps per second: 134, episode reward: 58.004, mean reward:  0.058 [-21.500, 23.964], mean action: 1.314 [0.000, 3.000],  loss: 8.508068, mae: 22.530595, mean_q: 25.537361, mean_eps: 0.277597
 121901/150000: episode: 901, duration: 7.886s, episode steps: 1000, steps per second: 127, episode reward: -11.128, mean reward: -0.011 [-21.814, 14.587], mean action: 1.711 [0.000, 3.000],  loss: 8.809993, mae: 2

 143771/150000: episode: 930, duration: 2.478s, episode steps: 345, steps per second: 139, episode reward: 236.485, mean reward:  0.685 [-14.019, 100.000], mean action: 1.522 [0.000, 3.000],  loss: 6.165199, mae: 20.828962, mean_q: 27.483780, mean_eps: 0.138412
 144713/150000: episode: 931, duration: 7.255s, episode steps: 942, steps per second: 130, episode reward: 140.280, mean reward:  0.149 [-20.089, 100.000], mean action: 1.304 [0.000, 3.000],  loss: 4.109369, mae: 20.768226, mean_q: 27.566711, mean_eps: 0.134551
 145456/150000: episode: 932, duration: 5.700s, episode steps: 743, steps per second: 130, episode reward: 174.815, mean reward:  0.235 [-20.721, 100.000], mean action: 1.857 [0.000, 3.000],  loss: 4.049153, mae: 20.871992, mean_q: 27.780278, mean_eps: 0.129496
 145878/150000: episode: 933, duration: 3.054s, episode steps: 422, steps per second: 138, episode reward: 254.777, mean reward:  0.604 [-10.768, 100.000], mean action: 1.438 [0.000, 3.000],  loss: 4.593402, mae: 2

Wow! After only some minutes of training, we achieve great results!
The reason for this is, that keras-rl has implemented many optimization strategies (e.g the optimized replay buffer) which lead to a much faster convergence than our DQN implemented by hand

In [20]:
# After training is done, we save the final weights.
dqn.save_weights(f'OKv1_LunarLander_2x150000steps_y0,992_a0,001.h5f', overwrite=True)

In [ ]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=200, visualize=False, callbacks=[WandbCallback()])
env.close()

In [12]:
dqn.load_weights(f'OKv1_LunarLander_2x150000steps_y0,992_a0,001.h5f')

In [14]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 255.760, steps: 199
Episode 2: reward: 228.585, steps: 346
Episode 3: reward: 246.992, steps: 190
Episode 4: reward: 270.068, steps: 192
Episode 5: reward: 254.709, steps: 303
